In [5]:
#import findspark
#findspark.init("/usr/local/spark")
import pyspark
import os
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
import re
import collections 
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
#import seaborn as sns
from pyspark.sql.functions import rand, randn
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.sql.types import IntegerType
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

In [6]:
conf=SparkConf().setMaster("local")
sc.stop()
sc=SparkContext(conf=conf)
sqlContext = SQLContext(sc)

# ColStat Function

#### Random Dataset

In [7]:
import numpy as np
#df = sqlContext.range(0, 10).withColumn('rand1', rand(seed=10)).withColumn('rand2', rand(seed=27))

from pyspark.mllib.stat import Statistics

mat = sc.parallelize(
    [np.array([1.0, 10.0, 100.0]), np.array([2.0, 20.0, 200.0]), np.array([3.0, 30.0, 300.0])])

#mat = sc.parallelize(
#    [sqlContext.range(0, 10).withColumn('rand1', rand(seed=10)), sqlContext.range(0, 10).withColumn('rand2', randn(seed=10))]
#)  # an RDD of Vectors

# Compute column summary statistics.
summary=Statistics.colStats(mat)
for i in range(3):
    print("\n"+ "Mean for column %(n)s is %(s)s" % {'n': int(i+1), 's': summary.mean()[i]} )
    print("Variance for column %(n)s is %(s)s" % {'n': int(i+1), 's': summary.variance()[i]} )
    print("Number of non-zeros for column %(n)s is %(s)s" % {'n': int(i+1), 's': summary.numNonzeros()[i]} )# a dense vector containing the mean value for each column
    #print(summary.variance()[i])  # column-wise variance
    #print(summary.numNonzeros()[i])  # number of nonzeros in each column
  


Mean for column 1 is 2.0
Variance for column 1 is 1.0
Number of non-zeros for column 1 is 3.0

Mean for column 2 is 20.0
Variance for column 2 is 100.0
Number of non-zeros for column 2 is 3.0

Mean for column 3 is 200.0
Variance for column 3 is 10000.0
Number of non-zeros for column 3 is 3.0


#### Our Dataset

In [8]:
rdd=sc.textFile('/home/burak/train_sample2.txt')
rdd_split = rdd.map(lambda x: x.split("\t"))
df=rdd_split.toDF()

Py4JJavaError: An error occurred while calling o95.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/burak/train_sample2.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:204)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Opt

In [ ]:

for i in range(len(df.columns)):
    df = df.withColumn("_"+str(i+1), df["_"+str(i+1)].cast(IntegerType()))
    
df_new =df.select("_1", "_2", "_3", "_4", "_5", "_6", "_7", "_8", "_9", "_10", "_11", "_12", "_13", "_14")
df_new=df_new.na.fill(0)

In [ ]:
from pyspark.mllib.linalg import Vectors
mat1=df_new.rdd.map(lambda row : Vectors.dense([item for item in row]))
summary=Statistics.colStats(mat1)
for i in range(len(df_new.columns)):
    print("\n"+ "Mean for column %(n)s is %(s)s" % {'n': int(i+1), 's': summary.mean()[i]} )
    print("Variance for column %(n)s is %(s)s" % {'n': int(i+1), 's': summary.variance()[i]} )
    print("Number of non-zeros for column %(n)s is %(s)s" % {'n': int(i+1), 's': summary.numNonzeros()[i]} )# 

## Correlations

In [ ]:
rdd1=sc.textFile('/home/burak/train_sample2.txt')
rdd1_split = rdd1.map(lambda x: x.split("\t"))

In [9]:
df=rdd1_split.toDF()

NameError: name 'rdd1_split' is not defined

In [ ]:
for i in range(len(df.columns)):
    df = df.withColumn("_"+str(i+1), df["_"+str(i+1)].cast(IntegerType()))
    
df_new =df.select("_1", "_2", "_3", "_4", "_5", "_6", "_7", "_8", "_9", "_10", "_11", "_12", "_13", "_14")
df_new=df_new.na.fill(0)

In [22]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

assembler = VectorAssembler(
  inputCols=[x for x in df_new.columns], outputCol="features")

assembled = assembler.transform(df_new)


In [10]:
assembled.select("features").rdd.take(10)

NameError: name 'assembled' is not defined

#### As it seen we transformed our data set to vectorized dense and sparse vector data type

In [31]:
r1 = Correlation.corr(assembled, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

Pearson correlation matrix:
DenseMatrix([[  1.00000000e+00,   1.05793760e-01,   3.43892970e-02,
               -1.73036823e-03,  -5.47143906e-02,  -7.61936253e-02,
               -6.13991220e-02,   8.85211203e-02,  -1.98359199e-02,
                1.99188471e-02,   1.97233541e-01,   1.72041424e-01,
                3.28722890e-02,  -3.49612313e-02],
             [  1.05793760e-01,   1.00000000e+00,   2.90884508e-02,
                5.31573539e-03,   6.19514288e-02,  -6.39554475e-02,
               -5.51209479e-02,   4.68818352e-01,   1.66518948e-02,
                4.85451193e-02,   4.27889568e-01,   2.49332850e-01,
                8.38467677e-02,  -5.22343654e-03],
             [  3.43892970e-02,   2.90884508e-02,   1.00000000e+00,
               -9.10402700e-03,  -8.34786817e-02,  -8.08840712e-03,
               -1.05089887e-02,   1.40080961e-02,  -1.28803943e-02,
               -7.32862226e-03,   2.20151767e-02,   1.64339847e-02,
               -1.27215090e-03,  -1.34249389e-02],
   

In [32]:
r2 = Correlation.corr(assembled, "features", "spearman").head()
print("Spearman correlation matrix:\n" + str(r2[0]))

Spearman correlation matrix:
DenseMatrix([[ 1.        ,  0.21129819,  0.00859781, -0.10776546, -0.06546042,
              -0.16112493, -0.06627577,  0.22903443, -0.05978631,  0.00888983,
               0.19577371,  0.2295929 ,  0.05599314, -0.15922038],
             [ 0.21129819,  1.        ,  0.02889873,  0.08348082,  0.19385097,
              -0.61799393, -0.12016754,  0.48141287,  0.21028064,  0.04108718,
               0.93457238,  0.4379942 ,  0.17088045,  0.0511587 ],
             [ 0.00859781,  0.02889873,  1.        ,  0.04002237,  0.09939959,
              -0.02212945,  0.01410925,  0.00855099,  0.06417939,  0.03303829,
               0.02731779,  0.00949485,  0.00447007,  0.11444111],
             [-0.10776546,  0.08348082,  0.04002237,  1.        ,  0.52350619,
              -0.07434861,  0.09258774,  0.02823411,  0.38311021,  0.11718579,
               0.07529945,  0.02217507,  0.01518783,  0.52416839],
             [-0.06546042,  0.19385097,  0.09939959,  0.52350619,  1.  

Correlation means to a relationship or association between quantities. It is useful to express onw
quantity in terms of its relationship with others.

    1- Pearson Correlation Coefficient
Pearson Correlation Coefficient measures the statistical relationship or linear association
between two continuous variables. Pearson Correlation Coefficient gives information about
the magnitude or correlation of the relationship as well as the direction of the relationship.
Because it is based on the covariance method, it is known one of the best methods of
measuring the relationship between the variables of interest. In other words, this coefficient
determines the degree to which the relationship between the two variables can be defined
by a line. This collation technique can be used if at least one of the variables is distributed
near normal or normal.

    2- Spearman’s Rank Correlation
Spearman’s Rank Correlation can be used to measure the degree of association between two
variables. It can be defined as a specific Pearson ρ example applied to the ordinal variables.
Unlike Pearson, Spearman&#39;s correlation is not limited to linear relations. Instead, it measures
the monotonic association between two variables and is relies on the rank order of the
values. In other words, instead of comparing averages and variances, the Spearman
coefficient looks in the order of relative value for each variable. This makes it suitable for use
with continuous and discrete data. It can be used when both variables are not normally
distributed, and also it can be used when at least one of the variables is an ordinal variable.